In [1]:
#import libraries
import pandas as pd
import numpy as np
import nltk 
from nltk.classify.textcat import TextCat 
from matplotlib.pyplot import plot 
from datascience import *

In [3]:
#fuzzywuzzy
!pip3 install fuzzywuzzy[speedup]
!pip install python-Levenshtein

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#(py)torch
#!pip install torch

#langid (language identification)
!pip install langid
import langid

  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl (18 kB)
Processing ./.cache/pip/wheels/a8/da/22/2970e270912ba623ccac7d516b7411a820c8f2b4252463a605/python_Levenshtein-0.12.0-cp38-cp38-linux_x86_64.whl
Processing ./.cache/pip/wheels/c5/01/a4/0160c55074707b535a6757a541842817d530d8080ca943a107/langid-1.1.6-py3-none-any.whl


In [4]:
#DO NOT RUN! This code creates a traditional table as seen in the datascience package but takes up too much memory. 
#I suggest running the code block after this one that generates chunks of 10 for each row of data.

chatbots_data = Table.read_table("chatbots_data.csv")
chatbots_data

In [4]:
#dividing the data into chunks of 10, because of memory issues (kernel keeps dying)
#for example, this is what one chunk looks like
chunksize = 10
for chunk in pd.read_csv("chatbots_data.csv", chunksize = chunksize):
    print(chunk.shape)
    print("="*66)
    print(chunk.head(2))
    print("="*66)
    break
    
#appending chunks to chatbot_list, an empty array
chatbot_list = []
chunksize = 10
for chunk in pd.read_csv("chatbots_data.csv", chunksize = chunksize):
    chatbot_list.append(chunk)

#Combine all the chunks into one dataframe
chatbots_data_chunks = pd.concat(chatbot_list, axis=0)
chatbots_data_chunks = pd.DataFrame(data = chatbots_data_chunks)

(10, 7)
              Created On     Flow Name  Merged Code  \
0  5/27/2020 10:31:06 PM    OB_NewUser          NaN   
1   5/27/2020 7:38:52 PM  HLP_Feedback          NaN   

                                             Message    Org Name  \
0                                            Kya kru   Chhaa Jaa   
1  I like you\nAnd I think improve not only about...  Big Sis V3   

   Referred Flow                                  Uuid  
0            NaN  915617a9-c0ae-47fc-8df0-54d72d4f41d9  
1            NaN  148d099c-54eb-4182-961c-068eeda96691  


In [5]:
#cleans the "Message" column
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"").replace('_',"")
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

chatbots_data_chunks['Message']=chatbots_data_chunks['Message'].map(lambda s:preprocess(s)) 

In [14]:
chatbots_data_chunks

,Created On,Flow Name,Merged Code,Message,Org Name,Referred Flow,Uuid
0,5/27/2020 10:31:06 PM,OB_NewUser,NaN,kya kru,Chhaa Jaa,NaN,915617a9-c0ae-47fc-8df0-54d72d4f41d9
1,5/27/2020 7:38:52 PM,HLP_Feedback,NaN,like nand think improve sex topics nbut best,Big Sis V3,NaN,148d099c-54eb-4182-961c-068eeda96691
2,5/27/2020 7:07:28 PM,OB_NewUser,NaN,nan,Chhaa Jaa,NaN,b05c8625-c18e-4f3a-9054-f7260f2541c5
3,5/27/2020 6:34:17 PM,HLP_Feedback,NaN,nihaiti bakwaas,Chhaa Jaa,NaN,afd1ead9-90b8-40c5-8fd6-b543a9a6067b
4,5/27/2020 5:53:14 PM,HLP_Feedback,NaN,mujhe apse bhut kuch seekhne mila,Chhaa Jaa,NaN,ddb1be9b-623d-4892-88c1-e441bf983103
...,...,...,...,...,...,...,...
82132,10/22/2020 4:00:18 AM,Lookup_unhandled,nullnull,topic,Big Sis V3,NEW MENU_heart of big sis,19491afa-307d-4ca2-b04a-89f5f21651fb
82133,10/22/2020 3:42:48 AM,Lookup_unhandled,nullnull,thank,Big Sis V3,FP_MS1_MSSA,cf79a515-0e43-480f-9885-3c90eac8fd86
82134,10/22/2020 3:41:25 AM,Lookup_unhandled,nullnull,,Big Sis V3,xplore,6901e04c-929b-4ab9-bd8f-20c17a39c823
82135,10/22/2020 3:40:57 AM,Lookup_unhandled,nullnull,,Big Sis V3,xplore,6901e04c-929b-4ab9-bd8f-20c17a39c823


In [5]:
#View cleaned data, all 80,000+ rows

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
chatbots_data_chunks

In [13]:
#collocation finder....might scrap later
import nltk
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = BigramCollocationFinder.from_words(chatbots_data_chunks["Message"].values.reshape(-1, ))

#bigrams that appear 3+ times
finder.apply_freq_filter(2)

# return the 10 n-grams with the highest PMI (pointwise mutual information)
finder.nbest(bigram_measures.pmi, 10)

[('always hurt sec', 'always hurt sec'),
 ('get horny', 'always get wet'),
 ('loves someone else', 'loves someone else'),
 ('reset', 'reset'),
 ('talk representative', 'agent'),
 ('needy', 'needy bee'),
 ('talk representative', 'cant underst'),
 ('goodbye conversation ends thank participating', 'talk representative'),
 ('cock', 'penis'),
 ('agent', 'cant underst')]

In [ ]:
#Backend portion...find the most common phrases 

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 

vectorizer = CountVectorizer(ngram_range = (3,3)) 
X1 = vectorizer.fit_transform(chatbots_data_chunks["Message"])  
features = (vectorizer.get_feature_names()) 
#print("\n\nFeatures : \n", features) <-- if you want to see all the phrases
#print("\n\nX1 : \n", X1.toarray())   <-- but I don't recommend running
  
# Applying TFIDF 
vectorizer = TfidfVectorizer(ngram_range = (3,3)) 
X2 = vectorizer.fit_transform(chatbots_data_chunks["Message"]) 
scores = (X2.toarray()) 
#print("\n\nScores : \n", scores) <--- I don't recommend running

In [6]:
# Getting top 20 ranking features 
sums = X2.sum(axis = 0) 
data1 = [] 
for col, term in enumerate(features): 
    data1.append( (term, sums[0,col] )) 
ranking = pd.DataFrame(data1, columns = ['term','rank']) 
words = (ranking.sort_values('rank', ascending = False)) 
# Alternatively, put "True" to see the most uncommon trigrams

common_phrases =  print("\n\nWords head : \n", words.head(20)) 
common_phrases



Words head : 
                              term        rank
25795          much services cost  306.000000
6879              change mind sex  123.004667
41998               thank big sis   76.751841
19235              know ready sex   65.209097
42125              thanks big sis   54.765816
16259                 hey big sis   53.509655
27913             normal sex hurt   52.912278
21907  long distance relationship   46.307602
18622     know boyfriend cheating   41.293563
36234              sex first time   38.243805
25742               much know sex   38.205654
12210              first time sex   35.978438
48523                yass big sis   33.000000
34387              right time sex   32.192785
22714           love really exist   28.510398
31543                poke big sis   28.000000
1762        anyone available chat   28.000000
20712                let talk sex   27.360898
23368         make boyfriend love   27.028650
36361  sex important relationship   26.208595


In [7]:
#backend part that allows fuzzy to work
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

choices = chatbots_data_chunks['Message'].unique()

In [8]:
# much services cost
# it appears that users are curious about the services the bots provide, as well as how much those services might cost
# however, fuzzywuzzy is also pulling "STIs" for "services" which could be an issue
process.extract("much services cost", choices, limit = 10, scorer = fuzz.token_sort_ratio)

[('much services cost', 100),
 ('much service cost', 97),
 ('cost services', 84),
 ('nthank services', 67),
 ('much cost', 67),
 ('ask phones services', 65),
 ('services offer', 62),
 ('services', 62),
 ('much know stis', 62),
 ('know much stis', 62)]

In [9]:
#change mind sex
process.extract("change mind sex", choices, limit = 10, scorer = fuzz.token_sort_ratio)

[('change mind sex', 100),
 ('chang mind sex', 97),
 ('change minde sex', 97),
 ('chañge miñd sex', 93),
 ('change mibd sex', 93),
 ('change mind sec', 93),
 ('change mind sez', 93),
 ('change mins sex', 93),
 ('cgange mind sex', 93),
 ('change myvmind sex', 91)]

In [10]:
#know ready sex
process.extract("know ready sex", choices, limit = 10, scorer = fuzz.token_sort_ratio)

[('know ready sex', 100),
 ('know read sex', 96),
 ('know yiu ready sex', 88),
 ('know ready sleep', 87),
 ('know ready', 83),
 ('ready know', 83),
 ('know ready start sex', 82),
 ('would know ready sex', 82),
 ('need know sex', 81),
 ('know oral sex', 81)]

In [30]:
# Plan ahead:
# Cluster data with Levenshtein distance: i.e., "change myvmind sex" is matched to "change mind sex"
# Find a better language identification algorithm, so bots don't mistake different languages as mispelled words
# which users are asking the most about which subject?

In [31]:
nltk.download('crubadan')
#N-Gram based categorization, identifying languages...which is very off...

tc = TextCat()
chatbots_data_chunks["Message"][0:5].apply(tc.guess_language)

[nltk_data] Downloading package crubadan to /opt/conda/nltk_data...
[nltk_data]   Unzipping corpora/crubadan.zip.


0    kac
1    sun
2    min
3    tgl
4    sun
Name: Message, dtype: object

In [7]:
import numpy as np
from scipy.sparse import csr_matrix
#!pip install sparse_dot_topn
import sparse_dot_topn.sparse_dot_topn as ct

Processing ./.cache/pip/wheels/4a/64/25/2e1cedbf7513fecda8d396a0431f3a1e217f45f1ab565e8ce8/sparse_dot_topn-0.2.9-cp38-cp38-linux_x86_64.whl


In [8]:
# backend string matching
from sklearn.feature_extraction.text import TfidfVectorizer

def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

message = chatbots_data_chunks["Message"]
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(message)

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
    idx_dtype = np.int32
    nnz_max = M*ntop
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)
    return csr_matrix((data,indices,indptr),shape=(M,N))

import time
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.8)
t = time.time()-t1

def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similarity': similairity})

In [9]:
#string-matching
matches_df = get_matches_df(matches, message, top=100000)
matches_df = matches_df[matches_df['similarity'] < 0.99999] # Remove all exact matches
matches_df.sample(50)

,left_side,right_side,similarity
96002,body shaming,body shame,0.818368
93516,need sex videos,sex videos,0.844718
53440,long take get pregnancy,long take get pregnant,0.926623
63713,teacher,teach,0.846122
56674,infections,infection,0.963932
81324,makes sensitive breasts,sensitive breast,0.827204
85376,sex feel like pregnant,yes feel like pregnant,0.831833
91237,know baby,want know baby,0.806466
68530,hot break virginity,one break virginity,0.836577
99539,relationship complicated,complicated relationships,0.837706


In [ ]:
#Convert Pandas dataframe to Table (where you can use table.select(...)). Not suggested -- takes up lots of memory
chatbots_data = Table.from_df(chatbots_data_chunks, keep_index=False)
chatbots_data